In [1]:
import os
os.chdir("../../")

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

In [3]:
!Rscript Toy.R

Loading required package: scater
Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: methods
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.m

In [ ]:
counts_df = pd.read_csv("./data/toy/toy.csv")
counts_df = counts_df.transpose()
counts_df.head()

In [ ]:
batches_df = pd.read_csv("./data/toy/toy_batch.csv")
batches_df['x'].value_counts()

In [ ]:
celltypes_df = pd.read_csv("./data/toy/toy_celltype.csv")
celltypes_df['x'].value_counts()

In [ ]:
counts_df.shape, batches_df.shape, celltypes_df.shape

In [ ]:
counts_adata = sc.AnnData(X=counts_df.values)
counts_adata.obs['study'] = batches_df.values
counts_adata.obs['celltype'] = celltypes_df.values
counts_adata.var_names = counts_df.columns
counts_adata

In [ ]:
counts_adata.write_h5ad("./data/toy/toy_count.h5ad")

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import surgeon as sg

In [ ]:
normalized_adata = counts_adata.copy()

In [ ]:
normalized_adata = sg.tl.normalize(normalized_adata, 'study', target_sum=1e4)
normalized_adata

In [ ]:
normalized_adata.X.min(), normalized_adata.X.max()

In [ ]:
normalized_adata.write_h5ad("./data/toy/toy_normalized.h5ad")